In [ ]:
oneailib_version = 1.13
oneailib_to_install = f"oneailib=={oneailib_version}" if oneailib_version else "oneailib"

In [ ]:
!pip install $oneailib_to_install --extra-index-url https://artifactory.us.caas.oneadp.com/artifactory/api/pypi/datacloud-datascience-pypi-local/simple

In [ ]:
dbutils.library.restartPython()

In [ ]:
from oneailib.embeddings.azure_openai import AzureOpenAIEmbedding
from oneailib.chat_models.azure_openai import AzureOpenAIChatModel
from oneailib.document_loaders.sql import SQLLoader
from oneailib.core.chat_models.base import FunctionCalling, FunctionCallingParameters, FunctionCallingPropertiesDocumentKey
from oneailib.document_transformers.transformers import EmbedKeyTransformer
from oneailib.prompt.openai import OpenAIPromptTemplate
from oneailib.document_transformers.metadata_transformers import (
    DecomposeKeysTransformer,
    AddDocumentIDTransformer, 
    AddDocumentVersionTransformer
)
import os

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

print("✓ Imports successful")

In [ ]:
dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
dbutils.widgets.text("volume", "datacloud-dita-gen-volume/FAQ_update/", "Volume")

In [ ]:
CATALOG_NAME = dbutils.widgets.get("catalog")
SCHEMA_NAME = dbutils.widgets.get("schema")
VOLUME_NAME = dbutils.widgets.get("volume")

In [ ]:
# os.environ["SERVICE_ACCOUNT_NAME"] = "service-cred-datacloud-dmt-jc-dit"

scope_name = "mishrapr"
os.environ["AZURE_TENANT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_TENANT_ID")
os.environ["AZURE_CLIENT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_ID")
os.environ["AZURE_CLIENT_SECRET"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_SECRET")

# scope_name = "FAQ_API"
# os.environ["AZURE_TENANT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_TENANT_ID")
# os.environ["AZURE_CLIENT_ID"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_ID")
# os.environ["AZURE_CLIENT_SECRET"] = dbutils.secrets.get(scope=scope_name, key="AZURE_CLIENT_SECRET")

In [ ]:
# ============================================================================
# REPLACED: Now using faq_generation module (moved to faq_generation/prompts.py and faq_generation/schemas.py)
# ============================================================================
# This cell has been replaced by importing from the faq_generation module
# See: faq_update/faq_generation/prompts.py and faq_update/faq_generation/schemas.py

from faq_generation.prompts import FAQ_ANSWER_GENERATION_PROMPT
from faq_generation.schemas import FAQ_ANSWER_GENERATION_SCHEMA

# The prompt and schema are now imported rather than defined inline
# This makes the code more maintainable and reusable
answer_prompt_template = FAQ_ANSWER_GENERATION_PROMPT
answer_function_calling = FAQ_ANSWER_GENERATION_SCHEMA

print("✓ FAQ answer generation prompts and schemas imported from faq_generation module")

In [ ]:
# Configuration constants

text_chunk_field = "chunk_text"
question_field = "question"
embedding_model_name = "text-embedding-ada-002_2-pgo-amrs"
chat_model_name = "gpt-4o_2024-11-20-pgo-amrs"

In [ ]:
# Configuration
import os

# Check if we're running in Databricks
def is_databricks_environment():
    """Check if we're running in a Databricks environment."""
    return 'DATABRICKS_RUNTIME_VERSION' in os.environ

if is_databricks_environment():
    print("🔧 Databricks environment detected!")
    
    # Unity Catalog paths
    PROJECT_ROOT = Path(f"/Volumes/{CATALOG_NAME}/{SCHEMA_NAME}/{VOLUME_NAME}")
    DB_PATH = None  # Unity Catalog doesn't use file paths for databases
    DATA_DIR = PROJECT_ROOT / "data_ingestion" / "data"
    MARKDOWN_DIR = DATA_DIR / "markdown"
    USE_UNITY_CATALOG = True
    
    print(f"Unity Catalog Configuration:")
    print(f"  Catalog: {CATALOG_NAME}")
    print(f"  Schema: {SCHEMA_NAME}")
    print(f"  Volume: {VOLUME_NAME}")
    print(f"  Project Root: {PROJECT_ROOT}")
    print(f"  Data Directory: {DATA_DIR}")
    print(f"  Markdown Directory: {MARKDOWN_DIR}")
    print(f"  Database: Unity Catalog managed tables")
    
else:
    print("💻 Local environment detected")
    
    # Original local configuration
    PROJECT_ROOT = Path.cwd().parent if 'granular_impact' in str(Path.cwd()) else Path.cwd()
    DB_PATH = PROJECT_ROOT / "databases" / "my_database.db"
    DATA_DIR = Path("data_ingestion/data")
    MARKDOWN_DIR = DATA_DIR / "markdown"
    USE_UNITY_CATALOG = False
    
    print(f"Local File System Configuration:")
    print(f"  Project Root: {PROJECT_ROOT}")
    print(f"  Database: {DB_PATH}")
    print(f"  Data Directory: {DATA_DIR}")
    print(f"  Markdown Directory: {MARKDOWN_DIR}")

# Path verification
if USE_UNITY_CATALOG:
    # For Databricks, create directories if they don't exist
    try:
        DATA_DIR.mkdir(parents=True, exist_ok=True)
        MARKDOWN_DIR.mkdir(parents=True, exist_ok=True)
        print("\n✓ Unity Catalog directories created/verified")
    except Exception as e:
        print(f"\n❌ Error creating directories: {e}")
        print(f"Make sure you have created the Unity Catalog volume:")
        print(f"CREATE VOLUME IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.{VOLUME_NAME};")
        
else:
    # Original verification for local environment
    assert DB_PATH.exists(), f"Database not found: {DB_PATH}"
    assert DATA_DIR.exists(), f"Data directory not found: {DATA_DIR}"
    assert MARKDOWN_DIR.exists(), f"Markdown directory not found: {MARKDOWN_DIR}"
    print("\n✓ All paths verified")

In [ ]:
from oneailib.document_loaders.sql import SQLLoader

# Define your configuration parameters
content_md_field = "context"  # Replace with actual content column name

# Define the fields schema
fields_schema = {
    "question_id": "",
    "question": "",
    "content_checksum": "",
    "context": "",
    "status": "",
    "chunk_id": ""
}

query = f"""
SELECT
    q.question_id,
    q.question_text as question,
    qs.content_checksum,
    cc.chunk_text as context,
    cc.chunk_id
FROM {CATALOG_NAME}.{SCHEMA_NAME}.faq_questions q
LEFT JOIN {CATALOG_NAME}.{SCHEMA_NAME}.faq_question_sources qs
    ON q.question_id = qs.question_id
LEFT JOIN {CATALOG_NAME}.{SCHEMA_NAME}.content_chunks cc
    ON qs.content_checksum = cc.content_checksum
LEFT JOIN {CATALOG_NAME}.{SCHEMA_NAME}.faq_answers a
    ON q.question_id = a.question_id AND a.status = "active"
WHERE q.status = "active"
    AND qs.is_valid = 1
    AND cc.status = "active"
    AND a.answer_id IS NULL
ORDER BY q.question_id
"""

# Initialize the SQLLoader
sql_loader = SQLLoader(
    query=query,
    content_column=content_md_field,
    # fields_schema=fields_schema
)

# Load the documents
chunk_documents = sql_loader.load()

# Display results
print(f"Loaded {len(chunk_documents)} documents")
for i, doc in enumerate(chunk_documents[:3]):  # Show first 3 documents
    print(f"\nDocument {i+1}:")
    print(f"Content preview: {doc.page_content[:200]}...")
    print(f"Metadata: {doc.metadata}")

In [ ]:
# ============================================================================
# NOTE: Embedding generation is now handled internally by AnswerGenerator
# ============================================================================
# The new faq_generation module can handle embedding generation internally
# when you set use_embeddings=True in the constructor.

# If you're using the new AnswerGenerator (Option 1 in next cell), 
# embeddings will be generated automatically during generate_from_documents()

# For the original approach (Option 2), keep this cell as is:
embedding_model = AzureOpenAIEmbedding(
    engine_name=embedding_model_name,
)

text_chunk_embedding_transformer = EmbedKeyTransformer(
    key="context",
    embeddings=embedding_model
)

question_embedding_transformer = EmbedKeyTransformer(
    key=question_field,
    embeddings=embedding_model
)

# These will only be used if you uncomment Option 2 in the next cell
# text_chunk_embedded_docs = text_chunk_embedding_transformer.transform_documents(chunk_documents)
# question_embedded_docs = question_embedding_transformer.transform_documents(text_chunk_embedded_docs)

print("✓ Embedding transformers initialized (for backward compatibility)")

In [ ]:
# ============================================================================
# OPTION 1: Using the NEW faq_generation module (RECOMMENDED)
# ============================================================================

from faq_generation import AnswerGenerator

# Create answer generator
answer_generator = AnswerGenerator(
    chat_model_name=chat_model_name,
    embedding_model_name=embedding_model_name,
    temperature=0.0,
    max_tokens=4000,
    max_retries=10,
    seconds_between_retries=5,
    use_embeddings=True  # Generate embeddings for semantic search
)

print(f"✓ AnswerGenerator initialized with embeddings={answer_generator.use_embeddings}")

# ============================================================================
# OPTION 2: Original approach (kept for backward compatibility)
# ============================================================================
# Uncomment below if you want to use the original approach:

# # Create the Answer generator model
# answer_generator_model = AzureOpenAIChatModel(
#     engine_name=chat_model_name,
#     temperature=0.0,
#     max_tokens=4000,
#     max_retries=10,
#     seconds_between_retries=5,
#     prompt=answer_prompt_template,
#     function_calling=answer_function_calling,
# )
#
# # Create answer decomposer
# answer_decomposer = DecomposeKeysTransformer(
#     keys_to_decompose={"answer_for_each_question": ["answer", "context_indexes"]}
# )

# ============================================================================

In [ ]:
# ============================================================================
# Using NEW faq_generation module
# ============================================================================

# Generate answers using the AnswerGenerator module
print(f"Generating answers from {len(chunk_documents)} documents...")
answer_generated_documents = answer_generator.generate_from_documents(
    documents=chunk_documents,
    add_ids=True,
    add_versions=True,
    generate_embeddings=True  # Generate embeddings for questions and context
)

print(f"✓ Generated {len(answer_generated_documents)} answers")

# ============================================================================
# Original approach (if using Option 2 from previous cell)
# ============================================================================
# Uncomment if using the original approach:

# answer_generated_documents = answer_generator_model(question_embedded_docs)

In [ ]:
# ============================================================================
# NOTE: Decomposition is now handled internally by AnswerGenerator
# ============================================================================
# The new faq_generation module handles decomposition internally in the
# generate_from_documents() method, so we can skip this step.
# The answer_generated_documents already contains decomposed answers.

# If you're using the original approach (Option 2), uncomment this:
# decomposed_answer_documents = answer_decomposer.transform_documents(answer_generated_documents)

# For the new approach, just use the already-decomposed documents:
decomposed_answer_documents = answer_generated_documents

print(f"✓ Using {len(decomposed_answer_documents)} decomposed answer documents")

In [ ]:
from pyspark.sql import Row
import pyspark.sql.functions as F

# Add IDs and versions to documents
id_transformer = AddDocumentIDTransformer()
documents_with_ids = id_transformer.transform_documents(decomposed_answer_documents)

version_transformer = AddDocumentVersionTransformer()
documents_with_version = version_transformer.transform_documents(documents_with_ids)

# Convert to Spark DataFrame
df = spark.createDataFrame([doc.metadata for doc in documents_with_version])

# Get all unique keys from the metadata 
all_keys = set()
for doc in documents_with_version:
    if hasattr(doc, "metadata"):
        all_keys.update(doc.metadata.keys())

print(f"All unique keys in documents: {sorted(all_keys)}")

In [ ]:
answer_data = []
for i, a in df.toPandas().iterrows():
    answer_data.append({
        'answer_id': a.id,
        'question_id': a.question_id,
        'answer_text': a.answer,
        'status': 'active'
    })
df_answer = pd.DataFrame(answer_data)
sdf_answers = spark.createDataFrame(df_answer)

In [ ]:
# Map to faq_question_sources table
sources_data = []
for i, a in df.toPandas().iterrows():
    if a.content_checksum:
        sources_data.append({
            'answer_id': a.id,
            'content_checksum': a.content_checksum,
            'context_employed': a.context,
            # 'chunk_version': q.version,quest
            # 'chunk_content': q.chunk_text,
            'is_primary_source': True,
            'contribution_weight': 1.0,
            'is_valid': True,
        })

df_answer_sources = pd.DataFrame(sources_data)
sdf_answer_sources = spark.createDataFrame(df_answer_sources)

In [ ]:
sdf_answers.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.faq_answers")

In [ ]:
sdf_answer_sources.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.faq_answer_sources")

In [ ]:
spark.sql(
    f"SELECT * FROM {CATALOG_NAME}.{SCHEMA_NAME}.faq_answers"
).display()